# Most Popular Pages in 2021

We gather the 50 most popular pages in 2021 for the annual end of year Medium post. The phab task for this is [T295943](https://phabricator.wikimedia.org/T295943)

## Configuration Variables and Methodological Notes

We use `wmf.pageviews_hourly` to gather page views for 2021, with the year being configurable. In our case we gather it for the English Wikipedia, but also have a configuration variable to change that to any wiki for simplicity. Because we aggregate by `page_id` the view counts automatically incorporate views coming in through any redirects. We use the most recent snapshot of `wmf_raw.mediawiki_page` (again configurable) to resolve page IDs to page titles.

We aggregate and calculate the proportion of views on the mobile platform in order to enable filtering of the list. A common heuristic is to exclude pages with less than 10% or more than 90% of their views coming in through mobile as that is an indicator of automated traffic.

Starting with the 2021 overview, we also aggregated the proportion of views not having a referrer header set (`referer_class = "none"`). After examining the data we found that several pages about companie or entities (e.g. YouTube, Wikipedia, Google) had a significantly higher proportion of their views coming in that way, and we chose to exclude them from the list as well.

Note that the annual blog post is updated after the year has ended with pageview counts from [Topviews](https://pageviews.toolforge.org/topviews?project=en.wikipedia.org) from the Pageviews tool. Those view counts do *not* include redirects and can therefore significantly alter the total number of views. This might be most significant for pages that have a large number of redirects (e.g. [Characters of the Marvel Cinematic Universe](https://pageviews.toolforge.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=1&range=last-year&pages=Characters_of_the_Marvel_Cinematic_Universe) had [273 redirects in the article namespace pointing to it](https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/Characters_of_the_Marvel_Cinematic_Universe&hidetrans=1&hidelinks=1&limit=500) in early January 2022, and the pageviews for 2021 are 19.8M when taking them into account but only 3.5M without the redirects).

Lastly, we have a configuration variable for the number of pages to return. It's by default set to 100. Note that our heuristics described above end up removing a fairly large number of pages from the list, so setting it higher than the number of pages you're aiming to finally list will be useful.

In [1]:
from wmfdata import spark

In [3]:
## The year we're gathering data for
year = 2021

## Name of the project in wmf.pageview_hourly
wiki = 'en.wikipedia'

## Database name and snapshot matching the project in wmf_raw.mediawiki_page
wiki_db = 'enwiki'
snapshot = '2021-12'

## The number of pages we return
top_n = 100

In [4]:
## Kudos to Connie for creating last year's query, which I used as a starting point!
## Added mobile view percentage as that helps determine whether specific articles 
## are disqualified list due to "automated" traffic not being caught by other heuristics.
## Added percentage of "none" referrers per T268262#6664186

pageview_query = f'''
WITH v AS (
    SELECT
        page_id,
        SUM(view_count) AS pageviews,
        SUM(IF(access_method = "mobile web", view_count, 0)) AS num_mobile_pageviews,
        100.0 * SUM(IF(access_method = "mobile web", view_count, 0)) / SUM(view_count)
            AS percent_mobile_pageviews,
        SUM(IF(referer_class = "none", view_count, 0)) AS num_noreferrer_pageviews,
        100.0 * SUM(IF(referer_class = "none", view_count, 0)) / SUM(view_count)
            AS percent_none_referrer
    FROM wmf.pageview_hourly
    WHERE year = {year}
        AND project = "{wiki}"
        AND namespace_id = 0 -- only article pages
        AND agent_type = "user" -- ignore bots, spiders, etc
    GROUP BY page_id
    ORDER by pageviews DESC
    LIMIT {top_n}
), p AS (
    SELECT
        page_id,
        page_title,
        page_latest
    FROM wmf_raw.mediawiki_page
    WHERE wiki_db = "{wiki_db}"
    AND snapshot = "{snapshot}"
    AND page_id IS NOT NULL
    AND page_namespace = 0
    AND NOT page_is_redirect
)
SELECT
    p.page_title,
    v.*
FROM v
LEFT JOIN p
ON v.page_id=p.page_id
'''

In [5]:
pageview_data = spark.run(pageview_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [ ]:
pageview_data.sort_values('pageviews').tail(50)

In [7]:
pageview_data.to_csv('english-wikipedia-2021-top-pages-2022-01-10.csv')